In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import torch
import logging
from PIL import Image
from lang_sam import LangSAM

# 로깅 설정
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
log_messages = []  # 모든 로그를 저장할 리스트

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
log_messages.append(f"Using device: {device}")

# 모델 로드
model = LangSAM()

# 입력 및 출력 폴더 설정
input_folder = "./assets"
output_folder = "./results"
os.makedirs(output_folder, exist_ok=True)

# 폴더 내 모든 이미지 파일 찾기
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
log_messages.append(f"Found {len(image_files)} images in {input_folder}")

# 텍스트 프롬프트 설정 (도로 균열 및 포트홀 탐지)
text_prompts = ["pothole", "manhole"]

# 모든 이미지 처리
for image_file in image_files:
    try:
        image_path = os.path.join(input_folder, image_file)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)  # PIL 이미지를 NumPy 배열로 변환

        # 이미지 정보 출력
        image_height, image_width = image_np.shape[:2]
        log_messages.append(f"Processing {image_file} - Size: {image_width} x {image_height}")

        # 추론 시작 (GPU 사용 시 동기화)
        if device == "cuda":
            torch.cuda.synchronize()

        start_time = time.time()

        # 🔹 각각의 프롬프트를 개별적으로 실행
        results = []
        for text_prompt in text_prompts:
            result = model.predict([image_pil], [text_prompt])[0]
            results.append((text_prompt, result))  # (프롬프트, 결과) 저장

        if device == "cuda":
            torch.cuda.synchronize()

        end_time = time.time()
        inference_time = end_time - start_time
        log_messages.append(f"{image_file} - Inference Time: {inference_time:.3f} sec")

        # 여러 개의 결과를 합치기 위해 빈 리스트 생성
        all_boxes, all_masks, all_scores, all_labels = [], [], [], []

        for text_prompt, result in results:
            all_boxes.extend(result.get("boxes", []))
            all_masks.extend(result.get("masks", []))
            all_scores.extend(result.get("scores", []))
            all_labels.extend(result.get("labels", []))

        num_masks = len(all_masks)
        avg_score = np.mean(all_scores) if len(all_scores) > 0 else 0

        log_messages.append(f"{image_file} - Detected Masks: {num_masks}")
        log_messages.append(f"{image_file} - Average Confidence Score: {avg_score:.4f}")

        # GPU 메모리 사용량 확인
        if device == "cuda":
            gpu_memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB 단위
            log_messages.append(f"{image_file} - GPU Memory Used: {gpu_memory:.2f} MB")

        # 박스와 마스크를 원본 이미지에 표시
        overlay = image_np.copy()

        for i, (box, mask, score, label) in enumerate(zip(all_boxes, all_masks, all_scores, all_labels)):
            if score < 0.5:  # 신뢰도 낮은 박스는 무시
                continue

            # Bounding Box 좌표
            x1, y1, x2, y2 = map(int, box)

            # 박스 그리기
            cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 255, 0), 3)  # 초록색 박스

            # 마스크 그리기
            mask = (mask > 0.5).astype(np.uint8) * 255  # 이진화
            overlay[mask > 0] = (0, 0, 255)  # 빨간색 마스크

            # 라벨 텍스트 추가
            cv2.putText(
                overlay, f"{label} {score:.2f}", (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2
            )

        # 결과 저장
        output_path = os.path.join(output_folder, f"result_{image_file}")
        cv2.imwrite(output_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
        log_messages.append(f"Saved result to {output_path}")

        # GPU 메모리 정리
        if device == "cuda":
            torch.cuda.empty_cache()

    except Exception as e:
        log_messages.append(f"Error processing {image_file}: {str(e)}")

log_messages.append("All images processed successfully!")

# 최종 결과 프린트
print("\n".join(log_messages))


Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
P

이미지 양 옆 붙이기기

In [7]:
import os
import numpy as np
import cv2
import time
import torch
import logging
from PIL import Image
from lang_sam import LangSAM

# 로깅 설정
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
log_messages = []

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
log_messages.append(f"Using device: {device}")

# 모델 로드
model = LangSAM()

# 입력 및 출력 폴더 설정
input_folder = "./assets"
output_folder = "./results"
os.makedirs(output_folder, exist_ok=True)

def enhance_cracks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    denoised = cv2.GaussianBlur(enhanced, (5,5), 0)
    return denoised

def detect_road_surface(image, model):
    result = model.predict([Image.fromarray(image)], ["asphalt road surface"])[0]
    if len(result["masks"]) == 0:
        return np.ones(image.shape[:2], dtype=bool)
    road_mask = result["masks"][0] > 0.5
    return road_mask

# 텍스트 프롬프트 설정
text_prompts = [
    "surface cracks on road",
    "thin line cracks on asphalt",
    "manhole",
    "pothole"
]

# 이미지 처리
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
for image_file in image_files:
    try:
        image_path = os.path.join(input_folder, image_file)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)

        # 도로 표면 감지
        road_mask = detect_road_surface(image_np, model)
        enhanced_image = enhance_cracks(image_np)
        
        # 2x2 그리드 이미지 준비
        segmented_images = {
            "original": image_np.copy(),
            "crack": image_np.copy(),
            "manhole": image_np.copy(),
            "pothole": image_np.copy()
        }

        # 크랙 마스크를 저장할 배열 초기화
        surface_crack_mask = np.zeros(image_np.shape[:2], dtype=bool)
        thin_line_mask = np.zeros(image_np.shape[:2], dtype=bool)
        
        # 크랙 결과 처리
        crack_results = {}
        for text_prompt in text_prompts[:2]:
            result = model.predict([image_pil], [text_prompt])[0]
            crack_results[text_prompt] = result

        # Surface cracks 처리
        if "surface cracks on road" in crack_results:
            result = crack_results["surface cracks on road"]
            for box, mask, score in zip(result["boxes"], result["masks"], result["scores"]):
                if score < 0.5:
                    continue
                
                mask = mask > 0.5
                mask = mask & road_mask
                
                if np.sum(mask) > (0.3 * image_np.shape[0] * image_np.shape[1]):
                    continue

                surface_crack_mask |= mask
                x1, y1, x2, y2 = map(int, box)
                
                cv2.rectangle(segmented_images["crack"], (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(
                    segmented_images["crack"],
                    f"surface crack {score:.2f}",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 255, 255),
                    2
                )

        # Thin line cracks 처리
        if "thin line cracks on asphalt" in crack_results:
            result = crack_results["thin line cracks on asphalt"]
            for box, mask, score in zip(result["boxes"], result["masks"], result["scores"]):
                if score < 0.5:
                    continue
                
                mask = mask > 0.5
                mask = mask & road_mask
                
                if np.sum(mask) > (0.3 * image_np.shape[0] * image_np.shape[1]):
                    continue

                thin_line_mask |= mask
                x1, y1, x2, y2 = map(int, box)
                
                cv2.rectangle(segmented_images["crack"], (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(
                    segmented_images["crack"],
                    f"thin line crack {score:.2f}",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 255, 255),
                    2
                )

        # 크랙 마스크 적용 (색상 구분)
        overlap_mask = surface_crack_mask & thin_line_mask  # 겹치는 부분
        surface_only_mask = surface_crack_mask & ~thin_line_mask  # surface crack만 있는 부분
        thin_only_mask = thin_line_mask & ~surface_crack_mask  # thin line만 있는 부분

        # 색상 적용
        segmented_images["crack"][surface_only_mask] = (0, 0, 255)    # 빨간색 (surface만)
        segmented_images["crack"][thin_only_mask] = (255, 0, 0)       # 파란색 (thin line만)
        segmented_images["crack"][overlap_mask] = (255, 0, 255)       # 보라색 (겹치는 부분)

        # 맨홀과 포트홀 처리
        for text_prompt in text_prompts[2:]:
            result = model.predict([image_pil], [text_prompt])[0]
            target_image = "manhole" if "manhole" in text_prompt.lower() else "pothole"
            
            for box, mask, score in zip(result["boxes"], result["masks"], result["scores"]):
                if score < 0.15:
                    continue
                
                mask = mask > 0.5
                x1, y1, x2, y2 = map(int, box)
                
                cv2.rectangle(segmented_images[target_image], (x1, y1), (x2, y2), (0, 255, 0), 2)
                segmented_images[target_image][mask] = (0, 0, 255)
                
                cv2.putText(
                    segmented_images[target_image],
                    f"{text_prompt} {score:.2f}",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 255, 255),
                    2
                )

        # 2x2 그리드 생성
        top_row = np.hstack([segmented_images["original"], segmented_images["crack"]])
        bottom_row = np.hstack([segmented_images["manhole"], segmented_images["pothole"]])
        combined_image = np.vstack([top_row, bottom_row])

        # 결과 저장
        output_path = os.path.join(output_folder, f"grid_{image_file}")
        cv2.imwrite(output_path, cv2.cvtColor(combined_image, cv2.COLOR_RGB2BGR))
        log_messages.append(f"Saved grid result to {output_path}")

        if device == "cuda":
            torch.cuda.empty_cache()

    except Exception as e:
        log_messages.append(f"Error processing {image_file}: {str(e)}")

print("\n".join(log_messages))

2025-02-13 10:14:42,631 [INFO] Computing image embeddings for the provided images...
2025-02-13 10:14:42,639 [INFO] Image embeddings computed.
2025-02-13 10:14:42,769 [INFO] Computing image embeddings for the provided images...
2025-02-13 10:14:42,774 [INFO] Image embeddings computed.


Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks


2025-02-13 10:14:42,899 [INFO] Computing image embeddings for the provided images...
2025-02-13 10:14:42,904 [INFO] Image embeddings computed.
2025-02-13 10:14:43,034 [INFO] Computing image embeddings for the provided images...
2025-02-13 10:14:43,039 [INFO] Image embeddings computed.


Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks


2025-02-13 10:14:43,172 [INFO] Computing image embeddings for the provided images...
2025-02-13 10:14:43,177 [INFO] Image embeddings computed.


...

Predicted 1 masks
Predicting 1 masks
Predicted 1 masks


2025-02-13 10:15:58,284 [INFO] Computing image embeddings for the provided images...
2025-02-13 10:15:58,289 [INFO] Image embeddings computed.
2025-02-13 10:15:58,426 [INFO] Computing image embeddings for the provided images...
2025-02-13 10:15:58,431 [INFO] Image embeddings computed.


Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks


2025-02-13 10:15:58,562 [INFO] Computing image embeddings for the provided images...
2025-02-13 10:15:58,567 [INFO] Image embeddings computed.


Predicting 1 masks
Predicted 1 masks
Using device: cuda
Saved grid result to ./results/grid_202502_30_01226897440_8466bfd11056483698e082fced1b38ac.jpg
Saved grid result to ./results/grid_car.jpeg
Saved grid result to ./results/grid_202502_30_01226896371_3a7a24ca3fd043d3a3a764006dfdb6e6.jpg
Saved grid result to ./results/grid_fruits.jpg
Saved grid result to ./results/grid_food.jpg
Saved grid result to ./results/grid_202502_30_01225815940_49ce32fdb81c4403ab4b31a52cb5e54c.jpg
Saved grid result to ./results/grid_202502_30_01226895996_7e684ce29df64738a142414ef43421dc.jpg
Saved grid result to ./results/grid_202502_30_01225815573_9258c2be1044485cad5d34a403a3623d.jpg
Saved grid result to ./results/grid_202502_30_01226888252_d281bbdb27b54ee5b05ffb3809ae1ac1.jpg
Saved grid result to ./results/grid_202502_30_01226898276_fb948faba7a54194b974ba18fe35782a.jpg
Saved grid result to ./results/grid_202502_30_01226897516_602e7c1e7373432ba15817eda3b55289.jpg
Saved grid result to ./results/grid_202502_30_0